# Predykcja wyniku meczu CS:GO
CS:GO - https://pl.wikipedia.org/wiki/Counter-Strike:_Global_Offensive

### Dane:
Dane zostały pozyskane poprzez webscarping ogolnodostępnej strony : https://www.hltv.org/, zawierające wyniki oraz statystyki profesjonalnych meczów cs:go.
Skrypty potrzebne do pobrania danych znajduja się w osobnym pliku !!!!!!!!!!!!!(NAZWE WSTAW)

### Cel:
Okreslenie wyniku pojedynczego spotkania - która drużyna zwycięzy. Sprowadza się to do zadania 1 jezeli wygrywa pierwsza druzyna oraz 0 jezeli wygrywa 2 druzyna (klasyfikator binarny)

### Klasa problemu:
Jako że znamy wyniki spotkań(label) mozemy wykorzystać uczenie nadzorowane (supervised learning). Znana jest takze liczba klas (2)





In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import clear_output

Pobieramy dane wczesniej przygotowane po stronie 2 skryptu

In [2]:
df_data = pd.read_pickle('df_data_connected.pkl')

In [3]:
pd.set_option('display.max_columns', 500)
df_data.head(10)

,date,team1main,team2main,map,eventmain,link,score_team1main,score_team2main,date_time,eventdetails,map_number,match_type,score_team1details,score_team2details,score_total,team1details,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2details,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,map_name
0,2012-10-11,NiP,HastaLaVista,Train_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,3,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,2.0,Best of 3,16,3,2 - 0,NiP,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,HastaLaVista,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,Train_se
1,2012-10-11,NiP,HastaLaVista,Train_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,3,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,1.0,Best of 3,16,8,2 - 0,NiP,2,15,-,-,-,-,-,0,0,0,0,0,13,15,14,15,16,-,-,-,-,-,26 (0),21 (0),18 (0),17 (0),16 (0),1.86,1.30,1.09,1.02,0.90,f0rest,friberg,GeT_RiGhT,Xizt,Fifflaren,1.23,10,6,ct-color,HastaLaVista,3,9,-,-,-,-,-,0,0,0,0,0,17,19,19,21,22,-,-,-,-,-,20 (0),16 (0),16 (0),10 (0),8 (0),1.32,0.92,0.82,0.45,0.37,Pimp,zn0pe,cajunb,ruggah,MSL,0.78,5,3,Train_se
2,2012-10-11,NiP,HastaLaVista,Inferno_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,2.0,Best of 3,16,3,2 - 0,NiP,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,HastaLaVista,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,Inferno_se
3,2012-10-11,NiP,HastaLaVista,Inferno_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,1.0,Best of 3,16,8,2 - 0,NiP,2,15,-,-,-,-,-,0,0,0,0,0,13,15,14,15,16,-,-,-,-,-,26 (0),21 (0),18 (0),17 (0),16 (0),1.86,1.30,1.09,1.02,0.90,f0rest,friberg,GeT_RiGhT,Xizt,Fifflaren,1.23,10,6,ct-color,HastaLaVista,3,9,-,-,-,-,-,0,0,0,0,0,17,19,19,21,22,-,-,-,-,-,20 (0),16 (0),16 (0),10 (0),8 (0),1.32,0.92,0.82,0.45,0.37,Pimp,zn0pe,cajunb,ruggah,MSL,0.78,5,3,Inferno_se
4,2012-10-11,Red Dead,mousesports,Inferno_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,2,16,2012-10-11 20:00,ESL Pro Series Germany Winter Season 2012,1.0,Best of 1,16,2,NaN,mousesports,3,13,-,-,-,-,-,0,0,0,0,0,6,11,11,9,10,-,-,-,-,-,24 (0),17 (0),16 (0),13 (0),12 (0),2.08,1.30,1.25,1.11,0.98

##### 1. Usuwamy zduplikowane wiersze

In [4]:
df_data = df_data.drop_duplicates(['link'])

In [5]:
df_data

,date,team1main,team2main,map,eventmain,link,score_team1main,score_team2main,date_time,eventdetails,map_number,match_type,score_team1details,score_team2details,score_total,team1details,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2details,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,map_name
0,2012-10-11,NiP,HastaLaVista,Train_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,3,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,2.0,Best of 3,16,3,2 - 0,NiP,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,HastaLaVista,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,Train_se
2,2012-10-11,NiP,HastaLaVista,Inferno_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,2.0,Best of 3,16,3,2 - 0,NiP,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,HastaLaVista,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,Inferno_se
4,2012-10-11,Red Dead,mousesports,Inferno_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,2,16,2012-10-11 20:00,ESL Pro Series Germany Winter Season 2012,1.0,Best of 1,16,2,NaN,mousesports,3,13,-,-,-,-,-,0,0,0,0,0,6,11,11,9,10,-,-,-,-,-,24 (0),17 (0),16 (0),13 (0),12 (0),2.08,1.30,1.25,1.11,0.98,Troubley,stavros,approx,nex,strux1,1.34,13,3,t-color,Red Dead,0,4,-,-,-,-,-,0,0,0,0,0,17,16,17,17,16,-,-,-,-,-,18 (0),11 (0),7 (0),7 (0),4 (0),1.11,0.73,0.47,0.40,0.28,fl0w,cavy,stefan,kev1n,caLipo,0.60,2,0,Inferno_se
5,2012-10-10,mousesports,Tt Dragons,Mirage_ce,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-10 21:00,ESWC 2012 German Qualifier,2.0,Best of 3,8,16,0 - 2,Tt Dragons,0,12,-,-,-,-,-,0,0,0,0,0,19,17,19,22,17,-,-,-,-,-,22 (14),17 (7),15 (4),16 (8),5 (2),1.15,0.97,0.76,0.67,0.48,Kaiii,mT,MoRs.na,kemo,deathX,0.81,6,2,t-color,mousesports,2,12,-,-,-,-,-,0,0,0,0,0,16,14,18,13,14,-,-,-,-,-,23 (12),21 (11),20 (3),14 (2),16 (8),1.56,1.34,1.14,0.98,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7,Mirage_ce
7,2012-10-10,Tt Dragons,mousesports,Dust2_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16,2012-10-10 21:00,ESWC 2012 German Qualifier,2.0,Best of 3,8,16,0 - 2,Tt Dragons,0,12,-,-,-,-,-,0,0,0,0,0,19,17,19,22,17,-,-,-,-,-,22 (14),17 (7),15 (4),16 (8),5 (2),1.15,0.97,0.7

##### Ponizej liczba wierszy przed wstepna obróbka, sprawdzamy typy naszych danych

In [6]:
len(df_data.index)

66654

In [7]:
df_data.dtypes

date                   datetime64[ns]
team1main                      object
team2main                      object
map                            object
eventmain                      object
link                           object
score_team1main                object
score_team2main                object
date_time                      object
eventdetails                   object
map_number                    float64
match_type                     object
score_team1details             object
score_team2details             object
score_total                    object
team1details                   object
team1_clutches_won             object
team1_first_kills              object
team1_p_ADR_1                  object
team1_p_ADR_2                  object
team1_p_ADR_3                  object
team1_p_ADR_4                  object
team1_p_ADR_5                  object
team1_p_asists_1               object
team1_p_asists_2               object
team1_p_asists_3               object
team1_p_asis

##### 2. Większosc naszych danych jest typu object - zamieniamy je do prawidlowego typu

Dane : 
- date - data spotkanie - te same informacje mamy w date_time - USUWAMY
- team1main - (str) nazwa zespol 1
- team2main - (str) nazwa zespol 2 
- map - (str) nazwa mapy 
- eventmain - (str) nazwa eventu 
- link - (str) link do strony zawierajace informacje o spotkaniu, bez wartosci przy modelowaniu - USUWAMY
- score_team1main - (int) wynik 1 zespol
- score_team2main - (int) wynik 2 zespol
- date_time - (datetime) data oraz czas spotkanie 
- eventdetails - (str) nazwa eventu - nazwa powtorzona - USUWAMY 
- map_number - (int) - nr mapy dla calego meczu 
- match_type - (str) - typ spotkania (best of 3, best of 5)
- score_team1details & score_team2details - nazwy druzyn - USUWAMY
- score_total -  wynik dla calego meczu - zawiera dane ktore nie beda dostepne w sytuacji prognozy(dane z przyszłosci)-USUWANY
- team1details & team2details - nazwy druzyn - juz obecne w bazie - usuwamy
- team1_clutches_won & team2_clutches_won - (int) wygranych clutchy - odpowiednio dla 1 i 2 druzyny (w sytuacji przewagi jednej z druzyn
- team1_first_kills & team2_first_kills - (int) liczba otwierajacych zabojstw
- team1_p_ADR_1 - (float) Average Damage Ratio - srednia zadawanych obrazen tu dla 1 gracza dla 1 druzyny - reszta ADR analogicznie 
- team1_p_asists_1 (int) liczba asyst w zabojstwach - analogicznie dla pozostalych graczy
- team1_p_death_1 (int) liczba smierci - analogicznie dla pozostalych graczy
- team1_p_kast_1 (float)  procent rund w ktorych gracz zaliczyl zabojstwo lub asyste - analogicznie dla pozostalych graczy
- team1_p_kills_1 (int) liczba zabojstw - analogicznie dla pozostalych graczy
- team1_p_rating_1 - (float) - rating dla gracza ustalany prze HLTV - analogicznie dla pozostalych graczy
- team1_player_name_1 (str) - nick zawodnika
- team1_rating (float) rating dla calego zespolu ustalany prze HLTV - analogicznie dla pozostalych graczy
- team1_score_1half (int) - wynik 1 zespolu po 1 polowwie - analogicznie dla 2 zespolu oraz dla 2 polow
- team1side (str) - strona po ktorej rozpoczyna 1 zespol
- map name - nazwa mapy - powtorzone - USUWAMY




In [8]:
# usuwamy niepotrzebne kolumny

#powtorzone kolumny
df_data = df_data.drop(columns =['eventdetails', 'score_team1details','score_team2details','team1details','team2details','map_name', 'link'])

#nie bedziemy mieli tej danej w przyszlosci
df_data = df_data.drop(columns =['score_total'])

In [9]:
# dodatkowo usuwamy wiersze dla ktorych nie udalo sie pobrac danych szczegolowych
df_data = df_data.dropna()

In [10]:
# mozemy takze zawazyc ze dane niedostepne sa zapisane jako - oznacza to ze dane zostaly pobrane ale nie sa one dostepne
# np. team1_p_ADR_1
df_data.head(5)

,date,team1main,team2main,map,eventmain,score_team1main,score_team2main,date_time,map_number,match_type,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half
0,2012-10-11,NiP,HastaLaVista,Train_se,ESWC 2012 Nordic Qualifier,16,3,2012-10-11 20:00,2.0,Best of 3,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0
2,2012-10-11,NiP,HastaLaVista,Inferno_se,ESWC 2012 Nordic Qualifier,16,8,2012-10-11 20:00,2.0,Best of 3,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0
4,2012-10-11,Red Dead,mousesports,Inferno_se,ESL Pro Series Germany Winter Season 2012,2,16,2012-10-11 20:00,1.0,Best of 1,3,13,-,-,-,-,-,0,0,0,0,0,6,11,11,9,10,-,-,-,-,-,24 (0),17 (0),16 (0),13 (0),12 (0),2.08,1.30,1.25,1.11,0.98,Troubley,stavros,approx,nex,strux1,1.34,13,3,t-color,0,4,-,-,-,-,-,0,0,0,0,0,17,16,17,17,16,-,-,-,-,-,18 (0),11 (0),7 (0),7 (0),4 (0),1.11,0.73,0.47,0.40,0.28,fl0w,cavy,stefan,kev1n,caLipo,0.60,2,0
5,2012-10-10,mousesports,Tt Dragons,Mirage_ce,ESWC 2012 German Qualifier,16,8,2012-10-10 21:00,2.0,Best of 3,0,12,-,-,-,-,-,0,0,0,0,0,19,17,19,22,17,-,-,-,-,-,22 (14),17 (7),15 (4),16 (8),5 (2),1.15,0.97,0.76,0.67,0.48,Kaiii,mT,MoRs.na,kemo,deathX,0.81,6,2,t-color,2,12,-,-,-,-,-,0,0,0,0,0,16,14,18,13,14,-,-,-,-,-,23 (12),21 (11),20 (3),14 (2),16 (8),1.56,1.34,1.14,0.98,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7
7,2012-10-10,Tt Dragons,mousesports,Dust2_se,ESWC 2012 German Qualifier,7,16,2012-10-10 21:00,2.0,Best of 3,0,12,-,-,-,-,-,0,0,0,0,0,19,17,19,22,17,-,-,-,-,-,22 (14),17 (7),15 (4),16 (8),5 (2),1.15,0.97,0.76,0.67,0.48,Kaiii,mT,MoRs.na,kemo,deathX,0.81,6,2,t-color,2,12,-,-,-,-,-,0,0,0,0,0,16,14,18,13,14,-,-,-,-,-,23 (12),21 (11),20 (3),14 (2),16 (8),1.56,1.34,1.14,0.98,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7


In [11]:
df_data = df_data.replace('-', np.nan)

In [12]:
# kolumna z zabojstwami zawiera informacje w formacie 12 (2) - z czym 1 wartosc oznacza liczbe zabojstw a 2 oznacza liczbe headshotow 
# przeksztalcamy dane w taki sposob aby wyciagnac z danych tylko liczbe zabojstw
df_data.team1_p_kills_1.iloc[-1]

'7 (1)'

In [13]:
def get_kills(row_text):
    kills = row_text.split(" ")[0]
    return kills

lista_kolumn = ['team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5', 'team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team2_p_asists_1',
       'team2_p_asists_2', 'team2_p_asists_3', 'team2_p_asists_4',
       'team2_p_asists_5','team2_p_kills_1', 'team2_p_kills_2',
       'team2_p_kills_3', 'team2_p_kills_4', 'team2_p_kills_5']

for kolumna in lista_kolumn:
    df_data[kolumna] = df_data[kolumna].apply(lambda x: get_kills(x))

In [14]:
# kolumna z % rund w ktorych byla asysta lub zabojstwo - jest zapisana w formacie XX% - ktory jest formatem str - 
# wycinamy %
df_data.iloc[-1]['team1_p_kast_1']

'96.3%'

In [15]:
def remove_last_elment_from_str(cell):
    if isinstance(cell , str):
        cell=cell[:-1]
    return cell


column_list = ['team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5', 'team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5']

for column in column_list:
    df_data[column] = df_data[column].apply(lambda x: remove_last_elment_from_str(x))

In [481]:
# mozemy zmienic typ danych na prawidlowe

In [16]:
#niestety inplace nie dziala


# NAPRAW TA KONWERSJE NA STRING
# str_list = ['team1main','team2main','map','eventmain','link','match_type','team1_player_name_1',
#        'team1_player_name_2', 'team1_player_name_3', 'team1_player_name_4',
#        'team1_player_name_5','team1side','team2_player_name_1',
#        'team2_player_name_2', 'team2_player_name_3', 'team2_player_name_4',
#        'team2_player_name_5']


# def to_string(column):
#     return new_str = [str(x) for x in old_obj][0]

# for column in str_list:
#     df_data[column] = df_data[column].apply(str)



int_list = ['score_team1main' , 'score_team2main','map_number','team1_clutches_won', 'team1_first_kills','team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team1_p_death_1', 'team1_p_death_2',
       'team1_p_death_3', 'team1_p_death_4', 'team1_p_death_5','team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5','team1_score_1half',
       'team1_score_2half', 'team2_clutches_won', 'team2_first_kills','team2_p_asists_1',
       'team2_p_asists_2', 'team2_p_asists_3', 'team2_p_asists_4',
       'team2_p_asists_5','team2_p_death_1', 'team2_p_death_2',
       'team2_p_death_3', 'team2_p_death_4', 'team2_p_death_5','team2_p_kills_1', 'team2_p_kills_2',
       'team2_p_kills_3', 'team2_p_kills_4', 'team2_p_kills_5','team2_score_1half',
       'team2_score_2half']

df_data[int_list] = df_data[int_list].astype(int)

float_list = ['team1_p_ADR_1','team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5','team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5','team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5','team1_rating','team2_p_ADR_1', 'team2_p_ADR_2', 'team2_p_ADR_3',
       'team2_p_ADR_4', 'team2_p_ADR_5','team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5','team2_p_rating_1', 'team2_p_rating_2', 'team2_p_rating_3',
       'team2_p_rating_4', 'team2_p_rating_5','team2_rating' ]

df_data[float_list] = df_data[float_list].astype(float, inplace=True)

In [17]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65760 entries, 0 to 853
Data columns (total 91 columns):
date                   65760 non-null datetime64[ns]
team1main              65760 non-null object
team2main              65760 non-null object
map                    65760 non-null object
eventmain              65760 non-null object
score_team1main        65760 non-null int32
score_team2main        65760 non-null int32
date_time              65760 non-null object
map_number             65760 non-null int32
match_type             65760 non-null object
team1_clutches_won     65760 non-null int32
team1_first_kills      65760 non-null int32
team1_p_ADR_1          53631 non-null float64
team1_p_ADR_2          53631 non-null float64
team1_p_ADR_3          53628 non-null float64
team1_p_ADR_4          53628 non-null float64
team1_p_ADR_5          53631 non-null float64
team1_p_asists_1       65760 non-null int32
team1_p_asists_2       65760 non-null int32
team1_p_asists_3       65760 non

##### 3. przetwarzamy dane tak aby tworzyly potrzebna informacje :
- z danych wynik okreslamy ktora druzyna jest zwycieska
- czyscimy nazwe map
- zamieniamy nazwy turniejow - dopasowujemy wielkosc turnieju do nazwy
- zamien date godzine na pore dnia

In [18]:
# tworzymy kolumne mowiaca ktora druzyna wygrala
def who_win(row):
    if row.score_team1main >row.score_team2main:
        return 1
    else:
        return 0

df_data['winner'] = df_data.apply(lambda x : who_win(x) , axis=1)

 

In [19]:
## czyscimy nazwy map
df_data['map'].unique()

array(['Train_se', 'Inferno_se', 'Mirage_ce', 'Dust2_se', 'Nuke_se',
       'Cache', 'Dust_se', 'Mill_ce', 'Season', 'Dust2', 'Inferno',
       'Mirage', 'Nuke', 'Train', 'Train_ve', 'Overpass', 'Cobblestone',
       'nuke', 'mrg', 'cch', 'inf', 'd2', 'Tuscan', 'Vertigo'],
      dtype=object)

In [20]:
map_names_to_change = {'Train_se':'Train', 'Inferno_se':'Inferno', 'Mirage_ce':'Mirage', 'Dust2_se':'Dust2', 'Nuke_se':'Nuke',
                      'Dust_se':'Dust', 'Mill_ce':'Mill','nuke':'Nuke','mrg':'Mirage','cch':'Cache', 'inf':'Inferno', 'd2':'Dust2', 'Train_ve':'Train'}

df_data['map'] = df_data['map'].replace(map_names_to_change)
df_data['map'].unique()

array(['Train', 'Inferno', 'Mirage', 'Dust2', 'Nuke', 'Cache', 'Dust',
       'Mill', 'Season', 'Overpass', 'Cobblestone', 'Tuscan', 'Vertigo'],
      dtype=object)

In [407]:
#czysicmy nazwy turniejow/serii
print(len(df_data['eventmain'].unique()))
df_data['eventmain'].unique()

3082


array(['ESWC 2012 Nordic Qualifier',
       'ESL Pro Series Germany Winter Season 2012',
       'ESWC 2012 German Qualifier', ...,
       'NCS Minor - Denver - June 2019', 'Aorus League 2019 #2 Finals',
       'ESL Nationals CZSK Season 1 Finals'], dtype=object)

In [217]:
#dzielimy turnieje/serie na 3 grupy Major, other
#do major zaliczamy turnieje z major oraz turnieje z iem w nazwie (bez tych dodatko z nazwa iem)

In [285]:
list_of_events = df_data['eventmain'].unique()
len(list_of_events)

3082

In [286]:
## usuwamy liczby, male litery , usuwamy duplikaty
list2=[]
for element in list_of_events:
    result = re.sub(r'([0-9#])', '', element)
    list2.append(result)

In [287]:
list2 = [s.lower() for s in list2]
list_of_events = set(list2)

len(list_of_events)

2124

In [199]:
qualifier = [s for s in list_of_events if "qualifier" in s]
list_of_events_left = [x for x in list_of_events if x not in qualifier]
len(list_of_events_left)

1445

In [200]:
regional = [s for s in list_of_events_left if "regional" in s ]
list_of_events_left = [x for x in list_of_events_left if x not in regional]
len(list_of_events_left)

1387

In [201]:
major = [s for s in list_of_events_left if "major" in s ]
list_of_events_left = [x for x in list_of_events_left if x not in major]
len(list_of_events_left)

1364

In [202]:
iem = [s for s in list_of_events_left if "iem" in s and 'minor' not in s ]
list_of_events_left = [x for x in list_of_events_left if x not in iem]
len(list_of_events_left)

1355

In [216]:
regional = [s for s in list_of_events_left if "finals" in s  ]
len(regional)

251

In [209]:
list_of_events_left

['nordic championship season ',
 'dust.dk eoddset masters  finals',
 'nsg academy - february ',
 'taltech  - spring',
 'dreamhack bucharest ',
 'worten game ring - rock in rio ',
 'ecs season  north america',
 'winners league season  europe',
 'dreamhack zowie open summer ',
 'rising stars asia ',
 'cs:go super league  finals',
 'rising stars female',
 'ga.com december cup powered by faceit europe',
 'tiger masters season  qualification ',
 'esl uk premiership season ',
 'european championship ',
 'dreamhack open rio ',
 'rising stars cis ',
 'cshub series season ',
 'superliga final four',
 'sltv starseries ix finals',
 'esl pro series germany winter season ',
 'eswc  denmark',
 'prague challenge',
 'loot.bet hotshot series season  cis',
 'la copa  finals',
 'ecs season  north america pinnacle cup',
 'geeks gone wild ',
 'digitalityx',
 'cevo professional season ',
 'acer cybergamer premier league autumn  finals',
 'asus rog winter ',
 'dell gaming liga pro season  -  apr/',
 'operati

In [83]:
list_of_events[16]

'Swedish Championship 2012 LAN qual #2'

In [55]:
m=re.match('' , list_of_events[0])

In [99]:
list2=[]
for element in list_of_events:
    result = re.sub(r'([0-9#])', '', element)
    list2.append(result)

In [100]:
list2

['ESWC  Nordic Qualifier',
 'ESL Pro Series Germany Winter Season ',
 'ESWC  German Qualifier',
 'ESWC  UK Qualifier',
 'GoCS:GO Cup ',
 'ESL Pro Series Poland Season ',
 'DreamHack Winter  Danish Qual.',
 'ESL Major Series Winter Season ',
 'CyberGamer Invite League Season ',
 'ESWC ',
 'DreamHack Winter ',
 'THOR Open Qualifier ',
 'DreamHack Winter  International Qual.',
 'SLTV StarSeries IV',
 'DreamHack Winter  Swedish Qual.',
 'DreamHack Winter  BYOC',
 'Swedish Championship  LAN qual ',
 'THOR Open Qualifier ',
 'AMD Sapphire CS:GO Invitational',
 'SteelSeries Finals',
 'NorthCon ',
 'RADEON Offensive by Damage',
 'THOR Open ',
 'IEM Poland International Qualifier',
 'Gaming.dk League by SteelSeries Season ',
 'Your road to Katowice ',
 'The Nordic Games  Danish qualifier',
 'Showmatch CS:GO',
 'SLTV StarSeries V',
 'Fnatic FragOut CS:GO League',
 'The Nordic Games ',
 'ESL Pro Series Poland Season  Finals',
 'Gaming.dk League by SteelSeries Season  Finals',
 'The Nordic Games  

In [167]:
list2 = (unique(list2))

In [168]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

In [131]:

list2 = [s.lower() for s in list2]
matching = [s for s in list2 if "qualifier" in s]

In [132]:
len(matching)

679

In [133]:
lista_www = [x for x in list2 if x not in matching]

In [139]:
matching2 = [s for s in lista_www if "major" in s and 'minor' not in s]

In [140]:
matching2

['esl major series winter season ',
 'king of majors',
 'esl major winter  spain',
 'eleague major ',
 'esl major league spring ',
 'pgl major krakow ',
 'esl major league summer ',
 'esl major league winter ',
 'faceit major ',
 'faceit major  showmatch',
 'gexcon  - ncs major']

In [145]:
matching2 = [s for s in lista_www if "iem" in s and 'minor' not in s ]

In [146]:
matching2

['iem gamescom ',
 'iem san jose ',
 'iem taipei ',
 'iem katowice ',
 'iem oakland ',
 'iem sydney ',
 'iem shanghai ',
 'iem chicago ',
 'iem sydney derby ']

In [163]:
matching2 = [s for s in lista_www if "regional" in s ]

In [164]:
matching2

['starladder regional minor championship cis',
 'pgl regional minor championship europe',
 'pgl kespa regional minor championship asia',
 'dreamhack regional minor championship europe at dh tours ',
 'mlg regional minor championship americas',
 'wesg  africa & middle east regional finals',
 'extremesland  china regional finals',
 'wesg  europe & cis regional finals',
 'wesg  north & south americas regional finals',
 'pgl regional minor championship asia - eleague major ',
 'pgl regional minor championship europe - eleague major ',
 'wesg  asia-pacific regional finals',
 'starladder regional minor championship cis - eleague major ',
 'rog masters  china regional finals',
 'rog masters  south asia regional finals',
 'rog masters  tw, hk & mo regional finals',
 'extremesland  thailand regional finals',
 'extremesland  anz regional finals',
 'extremesland  vietnam regional finals',
 'extremesland  indonesia regional finals',
 'extremesland  japan regional finals',
 'rog masters  indonesia 

In [226]:
#dizeliasmmy na 6-11 morning, 12-17 afternoon 18-23 Evening 0-5 Night

In [21]:
df_data['time_of_day'] = np.nan
df_data['time_of_day'][pd.to_datetime(df_data['date_time']).dt.hour.between(0,5)] = 'Night'
df_data['time_of_day'][pd.to_datetime(df_data['date_time']).dt.hour.between(6,11)] = 'Morning'
df_data['time_of_day'][pd.to_datetime(df_data['date_time']).dt.hour.between(12,17)] = 'Afternoon'
df_data['time_of_day'][pd.to_datetime(df_data['date_time']).dt.hour.between(18,23)] ='Evening'

C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Marcin\Anaconda3\

In [489]:
df_data

,date,team1main,team2main,map,eventmain,score_team1main,score_team2main,date_time,map_number,match_type,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,winner,time_of_day
0,2012-10-11,NiP,HastaLaVista,Train,ESWC 2012 Nordic Qualifier,16,3,2012-10-11 20:00,2,Best of 3,1,14,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,7,9,4,11,8,NaN,NaN,NaN,NaN,NaN,23,20,14,16,12,1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,17,16,18,18,NaN,NaN,NaN,NaN,NaN,11,8,6,7,7,0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,1,Evening
2,2012-10-11,NiP,HastaLaVista,Inferno,ESWC 2012 Nordic Qualifier,16,8,2012-10-11 20:00,2,Best of 3,1,14,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,7,9,4,11,8,NaN,NaN,NaN,NaN,NaN,23,20,14,16,12,1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,17,16,18,18,NaN,NaN,NaN,NaN,NaN,11,8,6,7,7,0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,1,Evening
4,2012-10-11,Red Dead,mousesports,Inferno,ESL Pro Series Germany Winter Season 2012,2,16,2012-10-11 20:00,1,Best of 1,3,13,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,6,11,11,9,10,NaN,NaN,NaN,NaN,NaN,24,17,16,13,12,2.08,1.30,1.25,1.11,0.98,Troubley,stavros,approx,nex,strux1,1.34,13,3,t-color,0,4,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,16,17,17,16,NaN,NaN,NaN,NaN,NaN,18,11,7,7,4,1.11,0.73,0.47,0.40,0.28,fl0w,cavy,stefan,kev1n,caLipo,0.60,2,0,0,Evening
5,2012-10-10,mousesports,Tt Dragons,Mirage,ESWC 2012 German Qualifier,16,8,2012-10-10 21:00,2,Best of 3,0,12,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,19,17,19,22,17,NaN,NaN,NaN,NaN,NaN,22,17,15,16,5,1.15,0.97,0.76,0.67,0.48,Kaiii,mT,MoRs.na,kemo,deathX,0.81,6,2,t-color,2,12,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,16,14,18,13,14,NaN,NaN,NaN,NaN,NaN,23,21,20,14,16,1.56,1.34,1.14,0.98,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7,1,Evening
7,2012-10-10,Tt Dragons,mousesports,Dust2,ESWC 2012 German Qualifier,7,16,2012-10-10 21:00,2,Best of 3,0,12,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,19,17,19,22,17,NaN,NaN,NaN,NaN,NaN,22,17,15,16,5,1.15,0.97,0.76,0.67,0.48,Kaiii,mT,MoRs.na,kemo,deathX,0.81,6,2,t-color,2,12,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,16,14,18,13,14,NaN,NaN,NaN,NaN,NaN,23,21,20,14,16,1.56,1.34,1.14,0.98,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7,0,Evening
9,2012-10-10,mousesports,ALTERNATE aTTaX,Dust2,ESL Pro Series Germany Winter Season 2012,16,7,2012-10-10 20:00,1,Best of 1,2,10,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,13,12,17,13,17,NaN,NaN,NaN,NaN,NaN,21,19,17,13,13,1.42,1.26,0.98,0.88,0.79,strux1,Troubley,stavros,approx,nex,1.06,8,8,ct-color,0,13,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,14,19,16,18,16,NaN,NaN,NaN,NaN,NaN,16,16,16,14,10,0.99,0.98,0.96,0.91,0.69,mnL,Rak,fel1x,SolEk,kRYSTAL,0.90,7,0,1

In [22]:
df_events =  pd.read_pickle('events.pkl')

In [491]:
# df_events = df_events.set_index('event_name')

In [23]:
df_events= df_events.drop(columns=['id','Maps'])

In [24]:
df_events.rename(columns={'event_name':'eventmain'}, inplace=True)

In [25]:
df_events['eventmain']=df_events['eventmain'].astype(str)
df_data['eventmain']=df_data['eventmain'].astype(str)

In [26]:
print(df_events['eventmain'].dtypes)
print(df_data['eventmain'].dtypes)

object
object


In [27]:
df_data= df_data.merge(df_events, on=['eventmain'],how='left')

In [28]:
df_data['type_of_event'] = df_data['type_of_event'].fillna('Online')

In [29]:
df_data.groupby('type_of_event').count()

,date,team1main,team2main,map,eventmain,score_team1main,score_team2main,date_time,map_number,match_type,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,winner,time_of_day
type_of_event,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BigEvents,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,2797,2797,2797,2797,2797,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,2797,2797,2797,2797,2797,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,2797,2797,2797,2797,2797,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,2797,2797,2797,2797,2797,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376,3376
Lan,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,15109,15109,15109,15109,15109,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,15109,15109,15109,15109,15109,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,15109,15109,15109,15109,15109,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,15109,15109,15109,15109,15109,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302,19302
Majors,584,584,584,584,584,584,584,584,584,584,584,584,355,355,355,355,355,584,584,584,584,584,584,584,584,584,584,355,355,355,355,355,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,355,355,355,355,355,584,584,584,584,584,584,584,584,584,584,355,355,355,355,355,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584
Online,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,38522,38522,38519,38519,38522,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,38531,38531,38531,38531,38529,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,38522,38519,38521,38524,38522,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,38531,38531,38531,38531,38531,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458,46458


In [30]:
df_data = df_data.sort_values('date_time')
df_data2_testowe = df_data.dropna()

In [31]:

columns_to_drop =  ['date','eventmain','date_time','team1_player_name_1','team1_player_name_2','team1_player_name_3'
                    ,'team1_player_name_4','team1_player_name_5','team2_player_name_1','team2_player_name_2','team2_player_name_3'
                    ,'team2_player_name_4','team2_player_name_5']

df_data2_testowe = df_data2_testowe.drop(columns=columns_to_drop)

In [32]:
columns_y=['winner']
columns_categorical = ['map',  'map_number', 'match_type','team1side','time_of_day', 'type_of_event' ]
columns_numerical = ['score_team1main', 'score_team2main', 
       'team1_clutches_won', 'team1_first_kills', 'team1_p_ADR_1',
       'team1_p_ADR_2', 'team1_p_ADR_3', 'team1_p_ADR_4', 'team1_p_ADR_5',
       'team1_p_asists_1', 'team1_p_asists_2', 'team1_p_asists_3',
       'team1_p_asists_4', 'team1_p_asists_5', 'team1_p_death_1',
       'team1_p_death_2', 'team1_p_death_3', 'team1_p_death_4',
       'team1_p_death_5', 'team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3',
       'team1_p_kast_4', 'team1_p_kast_5', 'team1_p_kills_1',
       'team1_p_kills_2', 'team1_p_kills_3', 'team1_p_kills_4',
       'team1_p_kills_5', 'team1_p_rating_1', 'team1_p_rating_2',
       'team1_p_rating_3', 'team1_p_rating_4', 'team1_p_rating_5',
       'team1_rating', 'team1_score_1half', 'team1_score_2half',
       'team2_clutches_won', 'team2_first_kills', 'team2_p_ADR_1',
       'team2_p_ADR_2', 'team2_p_ADR_3', 'team2_p_ADR_4', 'team2_p_ADR_5',
       'team2_p_asists_1', 'team2_p_asists_2', 'team2_p_asists_3',
       'team2_p_asists_4', 'team2_p_asists_5', 'team2_p_death_1',
       'team2_p_death_2', 'team2_p_death_3', 'team2_p_death_4',
       'team2_p_death_5', 'team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3',
       'team2_p_kast_4', 'team2_p_kast_5', 'team2_p_kills_1',
       'team2_p_kills_2', 'team2_p_kills_3', 'team2_p_kills_4',
       'team2_p_kills_5', 'team2_p_rating_1', 'team2_p_rating_2',
       'team2_p_rating_3', 'team2_p_rating_4', 'team2_p_rating_5',
       'team2_rating', 'team2_score_1half', 'team2_score_2half']




In [33]:
teams_list = {}
teams_list = set(df_data2_testowe['team1main'].unique())
teams_list.update(df_data2_testowe['team2main'].unique())

In [34]:
len(teams_list)

3560

In [447]:
teams_list = ['FaZe']

for idx,team in enumerate(teams_list):
    for column in columns_numerical:
        column_name_new= column+'_u'
        df_data2_testowe[column_name_new] = df_data2_testowe[column][df_data2_testowe['team1main']==team].shift(1).rolling(5).mean()
    print(idx)
    clear_output(wait=True)

0


In [437]:
df_data2_testowe['score_team1main_u'].unique()

array([ nan, 12.6, 13.6, 16. , 14. , 12. , 11.6, 11.2, 11.4, 15.6, 16.6,
       17. , 16.8, 15.8, 14.6, 13.4,  9.8,  8.2,  9.4, 14.4, 12.8, 15.4,
       14.8, 17.2, 10.6,  9. ,  9.6,  8.4,  8.6, 11. , 12.2, 13.2, 15.2,
       17.8, 18.4, 13. , 10.4, 13.8, 15. , 14.2, 17.4, 12.4,  8.8, 10. ,
       11.8, 16.2, 10.8, 16.4, 18.8, 19.4, 18.6,  7.6,  7.2,  6. ,  9.2,
        6.8,  7.4])

In [449]:
df_data2_testowe[df_data2_testowe['team1main']=='FaZe']

,team1main,team2main,map,score_team1main,score_team2main,map_number,match_type,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_rating,team2_score_1half,team2_score_2half,winner,time_of_day,type_of_event,score_team1main_u,score_team2main_u,team1_clutches_won_u,team1_first_kills_u,team1_p_ADR_1_u,team1_p_ADR_2_u,team1_p_ADR_3_u,team1_p_ADR_4_u,team1_p_ADR_5_u,team1_p_asists_1_u,team1_p_asists_2_u,team1_p_asists_3_u,team1_p_asists_4_u,team1_p_asists_5_u,team1_p_death_1_u,team1_p_death_2_u,team1_p_death_3_u,team1_p_death_4_u,team1_p_death_5_u,team1_p_kast_1_u,team1_p_kast_2_u,team1_p_kast_3_u,team1_p_kast_4_u,team1_p_kast_5_u,team1_p_kills_1_u,team1_p_kills_2_u,team1_p_kills_3_u,team1_p_kills_4_u,team1_p_kills_5_u,team1_p_rating_1_u,team1_p_rating_2_u,team1_p_rating_3_u,team1_p_rating_4_u,team1_p_rating_5_u,team1_rating_u,team1_score_1half_u,team1_score_2half_u,team2_clutches_won_u,team2_first_kills_u,team2_p_ADR_1_u,team2_p_ADR_2_u,team2_p_ADR_3_u,team2_p_ADR_4_u,team2_p_ADR_5_u,team2_p_asists_1_u,team2_p_asists_2_u,team2_p_asists_3_u,team2_p_asists_4_u,team2_p_asists_5_u,team2_p_death_1_u,team2_p_death_2_u,team2_p_death_3_u,team2_p_death_4_u,team2_p_death_5_u,team2_p_kast_1_u,team2_p_kast_2_u,team2_p_kast_3_u,team2_p_kast_4_u,team2_p_kast_5_u,team2_p_kills_1_u,team2_p_kills_2_u,team2_p_kills_3_u,team2_p_kills_4_u,team2_p_kills_5_u,team2_p_rating_1_u,team2_p_rating_2_u,team2_p_rating_3_u,team2_p_rating_4_u,team2_p_rating_5_u,team2_rating_u,team2_score_1half_u,team2_score_2half_u
12883,FaZe,Natus Vincere,Train,9,16,1,Best of 1,1,9,91.8,81.9,85.1,78.2,86.4,1,5,7,4,7,15,13,15,15,17,76.0,80.0,80.0,80.0,72.0,26,17,20,21,16,1.41,1.27,1.24,1.16,1.11,1.24,9,7,t-color,2,16,86.1,80.0,77.0,64.6,53.3,6,4,5,3,2,18,21,23,20,20,72.0,64.0,56.0,48.0,52.0,18,18,13,12,13,1.13,0.96,0.79,0.74,0.61,0.85,6,3,0,Afternoon,Lan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12966,FaZe,Luminosity,Cache,5,16,1,Best of 3,0,10,87.3,83.0,69.8,64.7,39.6,3,4,1,1,1,19,18,16,17,17,52.2,60.9,60.9,43.5,47.8,16,15,15,12,7,1.00,0.99,0.85,0.68,0.47,0.80,5,2,ct-color,1,13,104.4,108.8,76.8,73.3,43.9,3,4,1,6,0,14,15,11,12,13,73.9,73.9,87.0,78.3,78.3,21,22,18,13,13,1.50,1.43,1.38,1.18,0.98,1.29,10,6,0,Morning,Lan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12867,FaZe,Luminosity,Train,7,16,1,Best of 3,0,10,87.3,83.0,69.8,64.7,39.6,3,4,1,1,1,19,18,16,17,17,52.2,60.9,60.9,43.5,47.8,16,15,15,12,7,1.00,0.99,0.85,0.68,0.47,0.80,5,2,ct-color,1,13,104.4,108.8,76.8,73.3,43.9,3,4,1,6,0,1

In [432]:


for column in columns_categorical:
    df_data2_testowe = df_data2_testowe.join(pd.get_dummies(df_data2_testowe.column, prefix=column))

AttributeError: 'DataFrame' object has no attribute 'column'

In [504]:
df_data = df_data.reset_index()

In [517]:
df_data2_testowe[df_data2_testowe.team1main == 'NiP']

,team1main,team2main,map,score_team1main,score_team2main,map_number,match_type,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_rating,team2_score_1half,team2_score_2half,winner,time_of_day,type_of_event
13857,NiP,mousesports,Cache,19,15,1,Best of 1,2,15,92.9,89.0,71.3,48.1,42.5,5,3,4,2,4,24,21,23,26,25,67.6,79.4,67.6,61.8,50.0,30,29,19,14,11,1.28,1.27,0.93,0.56,0.53,0.91,8,7,t-color,2,19,112.1,85.9,68.2,59.3,50.4,3,5,5,4,5,19,20,21,22,21,79.4,79.4,58.8,79.4,70.6,39,27,22,17,14,1.73,1.36,0.96,0.96,0.79,1.16,7,8,1,Evening,Online
13856,NiP,mousesports,Overpass,16,14,1,Best of 1,4,18,101.5,107.7,65.2,60.0,58.2,6,6,6,3,1,23,21,20,21,18,73.3,76.7,73.3,63.3,50.0,29,26,15,17,16,1.38,1.35,0.96,0.87,0.80,1.07,10,6,ct-color,0,12,80.1,79.5,81.9,79.7,66.4,4,7,8,4,7,20,19,22,23,20,83.3,76.7,66.7,60.0,60.0,22,23,20,19,19,1.16,1.07,1.04,0.96,0.94,1.03,5,9,1,Evening,Online
13836,NiP,Natus Vincere,Mirage,16,10,1,Best of 1,4,17,97.9,92.2,72.5,78.7,56.9,2,5,5,5,1,15,15,13,21,15,92.3,76.9,88.5,65.4,73.1,28,23,15,18,16,1.66,1.37,1.22,1.04,1.04,1.27,10,6,ct-color,2,9,55.0,79.3,66.2,76.1,7.7,1,2,0,5,1,16,20,20,22,8,73.1,73.1,57.7,65.4,76.9,14,20,18,17,1,1.03,1.01,0.89,0.89,0.73,0.83,5,5,1,Evening,Online
14018,NiP,fnatic,Inferno,14,16,1,Best of 1,4,15,77.8,79.2,78.9,87.1,61.8,5,2,6,8,2,17,22,23,21,19,76.7,76.7,73.3,70.0,66.7,21,24,22,21,15,1.17,1.13,1.12,1.06,0.83,1.06,8,8,t-color,4,15,95.3,71.1,77.8,73.9,53.7,4,2,4,9,3,20,19,20,21,23,83.3,63.3,70.0,73.3,60.0,27,23,21,17,14,1.48,1.10,1.03,0.93,0.78,1.06,7,7,0,Afternoon,BigEvents
14017,NiP,fnatic,Inferno,14,16,1,Best of 1,4,15,77.8,79.2,78.9,87.1,61.8,5,2,6,8,2,17,22,23,21,19,76.7,76.7,73.3,70.0,66.7,21,24,22,21,15,1.17,1.13,1.12,1.06,0.83,1.06,8,8,t-color,4,15,95.3,71.1,77.8,73.9,53.7,4,2,4,9,3,20,19,20,21,23,83.3,63.3,70.0,73.3,60.0,27,23,21,17,14,1.48,1.10,1.03,0.93,0.78,1.06,7,7,0,Afternoon,Lan
14013,NiP,Natus Vincere,Overpass,13,16,1,Best of 1,1,10,96.3,94.7,77.1,53.4,67.9,6,1,4,5,6,22,18,24,17,25,72.4,79.3,69.0,69.0,62.1,27,25,17,14,16,1.25,1.23,0.94,0.79,0.78,1.00,8,5,ct-color,3,19,92.3,90.2,83.1,70.3,71.2,6,5,6,1,3,23,21,18,19,18,75.9,65.5,75.9,79.3,65.5,24,23,18,21,19,1.25,1.23,1.17,1.12,1.00,1.15,7,9,0,Afternoon,Lan
14014,NiP,Natus Vincere,Overpass,13,16,1,Best of 1,1,10,96.3,94.7,77.1,53.4,67.9,6,1,4,5,6,22,18,24,17,25,72.4,79.3,69.0,69.0,62.1,27,25,17,14,16,1.25,1.23,0.94,0.79,0.78,1.00,8,5,ct-color,3,19,92.3,90.2,83.1,70.3,71.2,6,5,6,1,3,23,21,18,19,18,75.9,65.5,75.9,79.3,65.5,24,23,18,21,19,1.25,1.23,1.17,1.12,1.00,1.15,7,9,0,Afternoon,BigEvents
14119,NiP,FaZe,Train,16,9,1,Best of 1,0,14,90.1,80.0,79.0,63.0,32.3,4,5,2,3,2,19,22,19,19,19,64.0,60.0,56.0,56.0,52.0,18,15,18,11,9,1.10,0.98,0.96,0.72,0.48,0.85,6,3,t-color,1,11,97.5,92.5,84.4,81.8,68.1,3,4,2,3,6,12,15,13,14,17,76.0,72.0,76.0,76.0,72.0,26,22,20,19,11,1.48,1.37,1.32,1.17,0.84,1.24,9,7,1,Evening,Online
14203,NiP,Virtus.pro,Cache,16,2,1,Best of 1,1,5,73.9,70.3,55.3,68.1,37.1,3,2,1,3,1,16,17,18,17,17,61.1,50.0,50

In [506]:
df_test = df_data[['score_team1main', 'team1main']].shift(1).rolling(5, on='team1main').mean()

In [36]:
df_data2_testowe = df_data2_testowe.sort_index()


In [553]:
df_data_test = df_data2_testowe.groupby('team1main', as_index=False).shift(1)

# UWAGA NIZEJ WAZNE SUPER

In [37]:
# temp = temp_master.groupby('player', sort=False)['won'].apply(lambda x : x.rolling(10, min_periods=1).sum().shift(1))\
#     .reset_index(drop=True)   


temp = df_data2_testowe.groupby('team1main', sort=False)[columns_numerical].apply(lambda x : x.rolling(5).mean().shift(1))

In [38]:
temp2 = temp.dropna()

In [40]:
temp_y = df_data2_testowe['winner']

In [41]:
temp3 = temp2.join(temp_y, how='left')

In [42]:
temp3.winner.sum()/temp3.winner.count()

0.529500756429652

In [43]:
df_data_test

NameError: name 'df_data_test' is not defined

In [44]:
df_data_test = df_data2_testowe.groupby('team1main', as_index=False)['score_team1main'].shift(1).groupby('team1main').rolling(5).mean()

KeyError: 'team1main'

In [544]:
df_data_test[]

11770     NaN
11771     NaN
11772     NaN
12483     NaN
12484     NaN
12485     NaN
12486     NaN
12532     NaN
12533     NaN
12568     NaN
12569     NaN
12570     NaN
12571     NaN
12572     NaN
12573     NaN
12574     NaN
12575     NaN
12576     NaN
12577     NaN
12578     NaN
12579     NaN
12580     NaN
12630     NaN
12633     NaN
12634     NaN
12635     NaN
12636     NaN
12637     NaN
12641     NaN
12642     NaN
         ... 
69690    14.2
69691    13.4
69692    14.2
69693    14.8
69694    14.8
69695    13.2
69696    13.0
69697    14.0
69698    13.6
69699    11.8
69700    13.4
69701    14.4
69702    14.4
69703    13.8
69704    15.6
69705    16.2
69706    16.2
69707    14.4
69708    13.8
69709    13.8
69710    13.2
69711    12.6
69712    11.8
69713    13.4
69714    13.0
69715    11.2
69716    11.8
69717    13.8
69718    13.8
69719    11.4
Name: score_team1main, Length: 56771, dtype: float64

In [548]:
df_data2_testowe

,team1main,team2main,map,score_team1main,score_team2main,map_number,match_type,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_rating,team2_score_1half,team2_score_2half,winner,time_of_day,type_of_event
11770,Envy,Titan,Train,11,16,3,Best of 3,4,12,86.3,89.2,76.4,28.0,32.9,3,2,3,2,4,19,20,20,7,13,70.4,63.0,74.1,88.9,77.8,23,19,21,6,6,1.16,1.07,1.07,1.02,0.83,1.01,7,4,ct-color,1,13,99.0,93.3,54.7,74.7,67.9,3,0,1,5,3,16,17,16,18,22,77.8,66.7,66.7,66.7,63.0,27,26,13,15,17,1.39,1.31,0.97,0.94,0.89,1.10,8,8,0,Evening,Online
11771,Titan,Envy,Cache,8,16,3,Best of 3,4,12,86.3,89.2,76.4,28.0,32.9,3,2,3,2,4,19,20,20,7,13,70.4,63.0,74.1,88.9,77.8,23,19,21,6,6,1.16,1.07,1.07,1.02,0.83,1.01,7,4,ct-color,1,13,99.0,93.3,54.7,74.7,67.9,3,0,1,5,3,16,17,16,18,22,77.8,66.7,66.7,66.7,63.0,27,26,13,15,17,1.39,1.31,0.97,0.94,0.89,1.10,8,8,0,Evening,Online
11772,Envy,Titan,Cobblestone,7,16,3,Best of 3,4,12,86.3,89.2,76.4,28.0,32.9,3,2,3,2,4,19,20,20,7,13,70.4,63.0,74.1,88.9,77.8,23,19,21,6,6,1.16,1.07,1.07,1.02,0.83,1.01,7,4,ct-color,1,13,99.0,93.3,54.7,74.7,67.9,3,0,1,5,3,16,17,16,18,22,77.8,66.7,66.7,66.7,63.0,27,26,13,15,17,1.39,1.31,0.97,0.94,0.89,1.10,8,8,0,Evening,Online
12483,ex-USSR,enable,Dust2,16,7,2,Best of 3,0,12,101.6,84.6,80.7,74.9,65.8,4,4,5,5,4,17,11,10,13,12,78.3,73.9,73.9,65.2,78.3,20,20,17,16,15,1.47,1.27,1.25,1.12,1.06,1.23,9,7,ct-color,1,11,82.6,64.7,59.0,76.0,34.2,5,1,1,2,0,17,18,18,18,17,60.9,60.9,65.2,52.2,39.1,14,13,15,13,7,1.00,0.95,0.82,0.78,0.49,0.81,6,1,1,Evening,Online
12484,enable,ex-USSR,Cache,16,12,2,Best of 3,0,12,101.6,84.6,80.7,74.9,65.8,4,4,5,5,4,17,11,10,13,12,78.3,73.9,73.9,65.2,78.3,20,20,17,16,15,1.47,1.27,1.25,1.12,1.06,1.23,9,7,ct-color,1,11,82.6,64.7,59.0,76.0,34.2,5,1,1,2,0,17,18,18,18,17,60.9,60.9,65.2,52.2,39.1,14,13,15,13,7,1.00,0.95,0.82,0.78,0.49,0.81,6,1,1,Evening,Online
12485,EYES ON U,enable,Inferno,9,16,1,Best of 1,3,14,114.6,82.4,71.9,76.8,62.4,9,3,5,6,5,18,15,15,13,15,80.0,76.0,84.0,84.0,68.0,28,22,16,18,13,1.59,1.30,1.29,1.24,0.89,1.26,7,9,t-color,2,11,69.8,77.6,63.6,68.8,59.0,3,2,2,4,3,17,20,16,22,22,76.0,64.0,68.0,72.0,48.0,17,18,14,14,13,0.99,0.91,0.89,0.77,0.61,0.83,8,1,0,Evening,Online
12486,Nostalgie,EYES ON U,Mirage,14,16,1,Best of 3,0,14,109.6,82.0,91.4,67.1,58.3,7,7,8,3,2,20,19,17,20,23,83.3,73.3,66.7,63.3,66.7,32,22,21,16,19,1.68,1.20,1.18,0.88,0.84,1.16,10,6,t-color,2,16,103.4,79.7,77.4,66.3,40.5,1,4,3,6,0,24,23,22,20,21,70.0,63.3,56.7,66.7,43.3,30,21,17,16,14,1.37,0.97,0.91,0.85,0.66,0.95,5,9,0,Evening,Online
12532,KoN Finland,KoN Sweden,Inferno,7,16,1,Best of 1,0,17,110.5,90.0,72.4,81.4,57.7,10,5,2,2,0,14,10,11,14,10,91.3,78.3,82.6,82.6,69.6,21,20,17,19,13,1.61,1.47,1.35,1.34,1.06,1.36,11,5,t-color,1,6,86.7,61.8,55.4,71.6,47.8,3,2,3,2,1,18,17,17,19,19,60.9,56.5,69.6,60.9,34.8,16,14,11,11,6,0.91,0.75,0.72,0.71,0.42,0.70,4,3,0,Evening,Online
12533,KoN Finland,KoN Norway,Mirage,16,5,1,Best of 1,0,6,77.4,68.6,60.6,58.8,59.6,0,2,2,5,2,18,16,17,18,17,66.7,52.4,52.4,

In [545]:
df_data_test = pd.DataFrame(df_data_test).reset_index()


In [546]:
df_data_test[df_data_test['index'] == 69010]

,index,score_team1main
56061,69010,14.4


In [458]:
df_data[df_data['team1main']=='NiP']

,date,team1main,team2main,map,eventmain,score_team1main,score_team2main,date_time,map_number,match_type,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,winner,time_of_day,type_of_event
23,2012-10-09,NiP,Publiclir.se,Dust2,ESWC 2012 Nordic Qualifier,16,12,2012-10-09 20:00,2,Best of 3,0,13,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,7,5,10,3,3,NaN,NaN,NaN,NaN,NaN,22,17,20,15,8,2.14,1.65,1.59,1.51,1.24,friberg,f0rest,Xizt,Fifflaren,GeT_RiGhT,1.63,14,2,ct-color,0,4,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,16,17,16,16,NaN,NaN,NaN,NaN,NaN,11,7,4,3,3,0.61,0.39,0.20,0.20,0.20,zneel,pita,pronax,rdl,lidde,0.32,1,0,1,Evening,Online
22,2012-10-09,NiP,Publiclir.se,Mirage,ESWC 2012 Nordic Qualifier,16,1,2012-10-09 20:00,2,Best of 3,0,13,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,7,5,10,3,3,NaN,NaN,NaN,NaN,NaN,22,17,20,15,8,2.14,1.65,1.59,1.51,1.24,friberg,f0rest,Xizt,Fifflaren,GeT_RiGhT,1.63,14,2,ct-color,0,4,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,16,17,16,16,NaN,NaN,NaN,NaN,NaN,11,7,4,3,3,0.61,0.39,0.20,0.20,0.20,zneel,pita,pronax,rdl,lidde,0.32,1,0,1,Evening,Online
1,2012-10-11,NiP,HastaLaVista,Inferno,ESWC 2012 Nordic Qualifier,16,8,2012-10-11 20:00,2,Best of 3,1,14,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,7,9,4,11,8,NaN,NaN,NaN,NaN,NaN,23,20,14,16,12,1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,17,16,18,18,NaN,NaN,NaN,NaN,NaN,11,8,6,7,7,0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,1,Evening,Online
0,2012-10-11,NiP,HastaLaVista,Train,ESWC 2012 Nordic Qualifier,16,3,2012-10-11 20:00,2,Best of 3,1,14,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,7,9,4,11,8,NaN,NaN,NaN,NaN,NaN,23,20,14,16,12,1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,17,16,18,18,NaN,NaN,NaN,NaN,NaN,11,8,6,7,7,0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,1,Evening,Online
166,2012-11-02,NiP,ProGaming.TD,Nuke,ESWC 2012,16,13,2012-11-02 11:45,1,Best of 1,1,21,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,21,16,22,20,NaN,NaN,NaN,NaN,NaN,24,27,22,22,12,1.31,1.24,1.24,1.03,0.62,friberg,Xizt,f0rest,GeT_RiGhT,Fifflaren,1.09,9,7,ct-color,0,8,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,21,22,21,20,23,NaN,NaN,NaN,NaN,NaN,25,24,17,16,14,1.21,1.17,0.81,0.75,0.67,rkz,zqkS,cogu,zakk,steel,0.92,6,7,1,Morning,BigEvents
165,2012-11-02,NiP,ProGaming.TD,Nuke,ESWC 2012,16,13,2012-11-02 11:45,1,Best of 1,1,21,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,21,16,22,20,NaN,NaN,NaN,NaN,NaN,24,27,22,22,12,1.31,1.24,1.24,1.03,0.62,friberg,Xizt,f0rest,GeT_RiGhT,Fifflaren,1.09,9,7,ct-color,0,8,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,21,22,21,20,23,NaN,NaN,NaN,NaN,NaN,25,24,17,16,14,1.21,1.17,0.81,0.75,0.67,rkz,zqkS,cogu,zakk,steel,0.92,6,7,1,Morning,Lan
162,2012-11-02,NiP,Ar

In [348]:
for idx,team in enumerate(teams_list):
    

Weekend Warriors
REXO
DUMSPIROSPERO
passions
ExcelsioR
Red Instinct
TTFU
Team Australia
Pompa Black
ex-eRa
Rulz
DISMAY
h4rr1-p0tt3r
GRIFFINS
Hi, I'm Billy Mays
Kheegui
Jaguar SA
Vault
FFI
Saints
Esi Pirmais
lol123
Não Tem Org
Evilvice
Zone
HNCE
4dimensioN
BOOT
T00EASY
NOTBAD
BRZINA
ONEGODLIKE
XRM
OneWay
Famou5
TOTOH
cs24h
Santos.DeX
duttdutt
Third Times a Charm
OnlineRol
NewTeam
cooL
liminka
Shukufuku Sunshine
Method
TG.FireStorm
eDGe
Modus Awperandi
Malhar
Power Ace
BUND
Itakera Classic
BERIH
Vision
Polbit Pogon
Gambit Academy
xoxo
Killing Your Idols
qup
IGG
Enigma
OESU
Apoka Academics
KHANS
Rão
Peeker's Advantage
SetToDestroyX
Sector 7
o9
Space Academy
EleMeNade
Aperture
X
ex-Zone
Cyberstorm
Wyvern Ladies
Washed up 17 year olds
BFAIM
Deceptors
average25iqbutheadshots
Orgless
Influx Nordic
Millennium
LeetPro fe
WinFiction
Team To Beat
mindplay
ASdffuf
TEAMMATE
SV
GD
MongrelMob
BLITZKRIEG
Ollebolle
Skillnaden
Dem Boyz
Valiance
No Quarter Cartel
Anti Eco Eco Club
Must Be Stronger
SITMEN

Ecuador
Ambush fe
Orgless fe
merry
Atlantic
NewStyle
TBA
Odense
inf!nite
Flexal
Shooting Monkeys
Ex-Rejected
Bonde da Stronda
Orange.Sphynx
PewPewX
GUTS
Online pro
GG1337
mousesports
Org Incoming
ex-NSPR
Team123
doktoRs
ORDER
tradigt
Balderisnor
COVE
Exdee
Sector One
ex-Atmosphere
Digital Wave
Napalm
Russia
ENIGMA
LeiSuRe
TRS
FX
IDM
AfterClap
8EASY
Synergy
Morituri
Baghlan
90s druglords
Dutch Prodigy
essketit
GUNRUNNERS
PEEKERS
Eyez-on
MOTHERS
1944
GoldenGitte
djakuyu
plusvee
LegendsNeverDie
Dignitas
dSc
Ze Pug Godz
Baltic Elite
Talentjes
heX
HIT AND RUN
Hollywood's Hooligans
JBellosNutyCrew
ex-Wisła Płock
EZG Reunion
Kazakhstan
Mask Off
OA
Iceberg
Prime5
Lowkey
Galatics
SemperX
ex-GameplayDNA
ex-3DMAX
Legacy of Desi Boys
Paradox Supremacy
Majestic
5Peaks
plan-B
TCC
OutPlayed
PRIDE Academy
WE HAWT
Mad Wolves
MDL All-Star freakazoid
WASDWASDwtfcantmove
hellslayerz
TripGhazi
ratoes
ex-Spotnet
Raiders
Albania
Off The Grid
GANSOS
PELAFUSTANES
Nodachi
NOPROB
23
Nice
SLAVGENT
TOASTBROT
Elysi

In [435]:


for idx,team in enumerate(teams):
    df_data['score_team1main'][df_data['team2main']==team].shift(1).rolling(10).mean()
    print(idx)
    clear_output(wait=True)

3299

KeyboardInterrupt: 

In [45]:
# sprawdzamy czy dane sa zbalansowane
df_data['winner'].sum()/df_data['winner'].count()

0.5120912220309811

1. wyczszczenie nazw map OK
2. zrobienie tego ktory wygral lub zremisowal OK
3. wielkosc turnieju
4. date_time - zamien na rano wieczorem poludnie w nocy
6. srednia_ratingow???  tu zaras sie cos wymysli
7. analiza ktory team na ktorym wygrywa czesciej - tzn ten co pierwszy z mapa\
8. średnie wyniki z ostatnich 10 spotkan dla kazdej z druzyn
9. srednie wyniki z ostaniego spotkania na tej mapie
10. srednie wyniki z ostatnich 10 spotkan na tej mapie
11. srednie wyniki ze spotkan 10 przeciwko
12. srednie wyniki ze spotkan na tej mapie wszystkie druzyny
13. srednie wyniki dla 1 strony, srednie wyniki dla 2 strony
14. srednie wyniki z pierwszej i 2 polowy ze wzgledu na zaczynana strone i mape


MOZE WYKORZYSTAC CZAS JAKO WAGE


PAMIETAJ o REQUIREMENTSACH

In [14]:
len(df_data.team1_player_name_1.unique())

5401

In [21]:
df_data[['score_team1main','team1main']].rolling(10, min_periods=1, on='team1main').sum()

,score_team1main,team1main
0,16.0,NiP
1,32.0,NiP
2,48.0,NiP
3,64.0,NiP
4,66.0,Red Dead
5,82.0,mousesports
6,98.0,mousesports
7,105.0,Tt Dragons
8,112.0,Tt Dragons
9,128.0,mousesports


In [31]:
df_data['score_team1main'][df_data['team1main'] == 'NiP'].shift(1).rolling(10).mean()

0          NaN
1          NaN
2          NaN
3          NaN
46         NaN
47         NaN
48         NaN
49         NaN
340        NaN
341        NaN
353       15.9
354       15.9
355       15.9
356       15.9
375       15.9
376       15.9
384       15.9
391       15.9
393       15.9
498       15.9
499       16.0
504       16.0
505       16.0
524       16.0
525       16.0
550       16.0
552       16.0
556       16.0
557       16.0
558       16.0
          ... 
141350    13.0
141351    13.1
141352    13.2
141455    14.0
141456    13.9
142427    12.7
142620    12.7
142621    12.7
142683    12.7
142684    12.5
142805    12.3
142806    11.9
142807    11.5
143389    11.1
143510    12.3
143534    13.5
143560    12.3
143561    12.3
143952    12.3
143953    12.5
144026    12.7
144027    13.4
144028    14.1
144464    14.8
144465    14.2
144466    13.6
144467    14.4
146439    14.0
146496    13.5
146497    12.5
Name: score_team1main, Length: 1405, dtype: float64

In [ ]:
df["mean"] = df["numeric_col"].shift(1).rolling(window=3).mean()

In [37]:
len(df_data['eventmain'].unique()) #dla eventu wez wytnij lata regular expresion - i potem sporobuj zobaczyc ile tego bedzie, wez majory do majorow reszte nizej

3073

In [38]:
print('nie umarl')

nie umarl


,date,team1main,team2main,map,eventmain,link,score_team1main,score_team2main,date_time,map_number,match_type,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,winner
0,2012-10-11,FAZE,HastaLaVista,Train,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,3,2012-10-11 20:00,2,Best of 3,1,14,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,7,9,4,11,8,NaN,NaN,NaN,NaN,NaN,23,20,14,16,12,1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,17,16,18,18,NaN,NaN,NaN,NaN,NaN,11,8,6,7,7,0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,1
2,2012-10-11,FAZE,HastaLaVista,Inferno,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-11 20:00,2,Best of 3,1,14,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,7,9,4,11,8,NaN,NaN,NaN,NaN,NaN,23,20,14,16,12,1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,17,16,18,18,NaN,NaN,NaN,NaN,NaN,11,8,6,7,7,0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,1
4,2012-10-11,FAZE,mousesports,Inferno,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,2,16,2012-10-11 20:00,1,Best of 1,3,13,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,6,11,11,9,10,NaN,NaN,NaN,NaN,NaN,24,17,16,13,12,2.08,1.30,1.25,1.11,0.98,Troubley,stavros,approx,nex,strux1,1.34,13,3,t-color,0,4,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,17,16,17,17,16,NaN,NaN,NaN,NaN,NaN,18,11,7,7,4,1.11,0.73,0.47,0.40,0.28,fl0w,cavy,stefan,kev1n,caLipo,0.60,2,0,0
5,2012-10-10,FAZE,Tt Dragons,Mirage,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-10 21:00,2,Best of 3,0,12,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,19,17,19,22,17,NaN,NaN,NaN,NaN,NaN,22,17,15,16,5,1.15,0.97,0.76,0.67,0.48,Kaiii,mT,MoRs.na,kemo,deathX,0.81,6,2,t-color,2,12,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,16,14,18,13,14,NaN,NaN,NaN,NaN,NaN,23,21,20,14,16,1.56,1.34,1.14,0.98,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7,1
7,2012-10-10,FAZE,mousesports,Dust2,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16,2012-10-10 21:00,2,Best of 3,0,12,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,19,17,19,22,17,NaN,NaN,NaN,NaN,NaN,22,17,15,16,5,1.15,0.97,0.76,0.67,0.48,Kaiii,mT,MoRs.na,kemo,deathX,0.81,6,2,t-color,2,12,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,16,14,18,13,14,NaN,NaN,NaN,NaN,NaN,23,21,20,14,16,1.56,1.34,1.14,0.98,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7,0
9,2012-10-10,FAZE,ALTERNATE aTTaX,Dust2,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,16,7,2012-10-10 20:00,1,Best of 1,2,10,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,13,12,17,13,17,NaN,NaN,NaN,NaN,NaN,21,19,17,1

In [49]:
df_data.columns

Index(['date', 'team1main', 'team2main', 'map', 'eventmain', 'link',
       'score_team1main', 'score_team2main', 'date_time', 'eventdetails',
       'map_number', 'match_type', 'score_team1details', 'score_team2details',
       'score_total', 'team1details', 'team1_clutches_won',
       'team1_first_kills', 'team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5', 'team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team1_p_death_1', 'team1_p_death_2',
       'team1_p_death_3', 'team1_p_death_4', 'team1_p_death_5',
       'team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5', 'team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5',
       'team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5', 'team1_player_name_1',
       'team1_player_name_2', 'team1_pl

In [49]:
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

from sklearn.metrics import accuracy_score

In [47]:
y = temp3.winner
X = temp3.drop(columns='winner')

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
names = np.array(["Naiwny Bayes", "Drzewo decyzyjne", "Regresja logistyczna", "SVM",'Las losowy random', 'Las losowy bagging'])
models = [[("model", MultinomialNB())],
         [("model", DecisionTreeClassifier())],
         [("scaler", MaxAbsScaler()), ("model", LogisticRegression())],
         [("scaler", MaxAbsScaler()),("model", SVC())],
         [("model", RandomForestClassifier())],
         [("model", BaggingClassifier())]]

param_grids = [{"model__alpha": [0.1, 1, 10], "model__fit_prior": [False, True]},
               {"model__criterion": ["gini", "entropy"], "model__min_samples_split": [2, 10, 100], "model__max_depth": [None, 2, 10, 100]}, 
               {"model__penalty": ["l1", "l2"], "model__C": [0.1, 1, 10]},
              [{"model__kernel": ["linear"]},
              {"model__kernel": ["rbf"]},
              {"model__kernel": ["poly"], "model__degree": [2, 3]}],
               {"model__n_estimators" : [100,50,20], "model__max_features" : [0.5]},
               {"model__max_samples" : [0.5], "model__bootstrap" : [True, False]}
              ]



uses=np.array([True, True, True, True, True, True])


if len(names) != len(models) or len(models) != len(param_grids):
    raise ValueError("Listy nie mają tej samej długości!")

best_models = []

for use, name, pipe, params in zip(uses, names, models, param_grids):
    print(f"Tunuje model: {name}")
    if use==False:
        print("pomijamy")
    else:
        pipeline = Pipeline(pipe)
        gs = GridSearchCV(estimator=pipeline, param_grid=params)
        gs.fit(X_train, y_train)
        for mean, std, param, fit_time, score_time in zip(gs.cv_results_["mean_test_score"],
                                    gs.cv_results_["std_test_score"],
                                    gs.cv_results_["params"],
                                    gs.cv_results_["mean_fit_time"],
                                    gs.cv_results_["mean_score_time"]):
            print(f"{param}:\n mean: {mean}, std: {std},\n fit_time: {fit_time}, score_time: {score_time}\n")
        best_models.append(gs.best_estimator_)


        
        
print("\n\nTestuję:")
for name, best_model in zip(names[uses], best_models):
    print(f"{name}: {accuracy_score(best_model.predict(X_test), y_test)}")

Tunuje model: Naiwny Bayes


C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'model__alpha': 0.1, 'model__fit_prior': False}:
 mean: 0.5180167286836498, std: 0.0072588843492418275,
 fit_time: 0.01733247439066569, score_time: 0.0040013790130615234

{'model__alpha': 0.1, 'model__fit_prior': True}:
 mean: 0.5225328371974685, std: 0.007690418937283357,
 fit_time: 0.01833375295003255, score_time: 0.003332932790120443

{'model__alpha': 1, 'model__fit_prior': False}:
 mean: 0.5180167286836498, std: 0.0072588843492418275,
 fit_time: 0.017999887466430664, score_time: 0.003668626149495443

{'model__alpha': 1, 'model__fit_prior': True}:
 mean: 0.5225328371974685, std: 0.007690418937283357,
 fit_time: 0.016999006271362305, score_time: 0.003334204355875651

{'model__alpha': 10, 'model__fit_prior': False}:
 mean: 0.5180167286836498, std: 0.00718113311193166,
 fit_time: 0.016998449961344402, score_time: 0.0033341248830159507

{'model__alpha': 10, 'model__fit_prior': True}:
 mean: 0.5226282479407182, std: 0.0076298110080345784,
 fit_time: 0.017001549402872723, score_time: 0.0

C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'model__criterion': 'gini', 'model__max_depth': None, 'model__min_samples_split': 2}:
 mean: 0.5136596380752473, std: 0.0009294049038706319,
 fit_time: 1.5513367652893066, score_time: 0.005999565124511719

{'model__criterion': 'gini', 'model__max_depth': None, 'model__min_samples_split': 10}:
 mean: 0.5111153515885889, std: 0.0009730024832731053,
 fit_time: 1.5326639811197917, score_time: 0.006333589553833008

{'model__criterion': 'gini', 'model__max_depth': None, 'model__min_samples_split': 100}:
 mean: 0.513627834494164, std: 0.004218502570866991,
 fit_time: 1.2703314622243245, score_time: 0.00566871960957845

{'model__criterion': 'gini', 'model__max_depth': 2, 'model__min_samples_split': 2}:
 mean: 0.5409153070635754, std: 0.0015469753531170973,
 fit_time: 0.15633257230122885, score_time: 0.0040026505788167315

{'model__criterion': 'gini', 'model__max_depth': 2, 'model__min_samples_split': 10}:
 mean: 0.5409153070635754, std: 0.0015469753531170973,
 fit_time: 0.1553321679433187, sc

C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be chang

{'model__C': 0.1, 'model__penalty': 'l1'}:
 mean: 0.5489934166587158, std: 0.001831952526656452,
 fit_time: 0.3836677074432373, score_time: 0.006999095280965169

{'model__C': 0.1, 'model__penalty': 'l2'}:
 mean: 0.5474350411856375, std: 0.0009802522028700966,
 fit_time: 0.4606659412384033, score_time: 0.007002035776774089

{'model__C': 1, 'model__penalty': 'l1'}:
 mean: 0.5469579874693891, std: 0.002184514805975816,
 fit_time: 17.445999542872112, score_time: 0.008335590362548828

{'model__C': 1, 'model__penalty': 'l2'}:
 mean: 0.5475940590910536, std: 0.002533130776546746,
 fit_time: 0.7676662604014078, score_time: 0.007334788640340169

{'model__C': 10, 'model__penalty': 'l1'}:
 mean: 0.5480711128073021, std: 0.0021081720470917512,
 fit_time: 61.02528381347656, score_time: 0.008000850677490234

{'model__C': 10, 'model__penalty': 'l2'}:
 mean: 0.5474350411856375, std: 0.0018007680777142403,
 fit_time: 1.3270000616709392, score_time: 0.007665793100992839

Tunuje model: SVM


C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Marcin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'a